In [1]:
%pip install pinyin

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import json
import requests
import pinyin
import csv
from tqdm import tqdm

In [3]:
data = {}
dateTable = {"第一天": 0, "第二天": 1, "第三天": 2, "第四天": 3, "第五天": 4, "第六天": 5}

with open('data/火车班次json数据', 'r', encoding = 'utf-8') as file:
    for line in file:
        currLine = json.loads(line)
        # print(currLine)
        currLineID = list(currLine["ticketInfo"].keys())[0]
        currLineStations = currLine["trainScheduleBody"]
        currLineMidSta = []
        for sta in currLineStations:
            # Time
            if sta["content"][3] == "起点站":
                staTime = sta["content"][4].split(":")
                staCost = 0
                staDist = 0
                currLineMidSta.append((int(staTime[0]), int(staTime[1]), staCost, staDist, sta["content"][1], currLineID))
            else:
                staTime = sta["content"][3].split(":")
                staCost = sta["content"][7]
                staDist = sta["content"][6].split("公里")
                currLineMidSta.append((int(staTime[0]) + 24 * dateTable[sta["content"][2]], int(staTime[1]), staCost, float(staDist[0]), sta["content"][1], currLineID))

        currLineMidSta = sorted(currLineMidSta)

        for ind in range(len(currLineMidSta) - 1):
            if currLineMidSta[ind][4] not in data:
                data[currLineMidSta[ind][4]] = {}
            if currLineMidSta[ind + 1][4] not in data:
                data[currLineMidSta[ind + 1][4]] = {}
            if currLineMidSta[ind + 1][1] < currLineMidSta[ind][1]:
                currMinute = 60 - currLineMidSta[ind][1] + currLineMidSta[ind + 1][1]
                currHour = currLineMidSta[ind + 1][0] - currLineMidSta[ind][0] - 1
            else:
                currMinute = currLineMidSta[ind + 1][1] - currLineMidSta[ind][1]
                currHour = currLineMidSta[ind + 1][0] - currLineMidSta[ind][0]
            if currLineMidSta[ind + 1][4] in data[currLineMidSta[ind][4]]:
                if currHour > data[currLineMidSta[ind][4]][currLineMidSta[ind + 1][4]][0]:
                    continue
                elif currHour == data[currLineMidSta[ind][4]][currLineMidSta[ind + 1][4]][0]:
                    if currMinute >= data[currLineMidSta[ind][4]][currLineMidSta[ind + 1][4]][1]:
                        continue
            
            currCost = currLineMidSta[ind + 1][2] - currLineMidSta[ind][2]

            currDist = currLineMidSta[ind + 1][3] - currLineMidSta[ind][3]

            currLineID = currLineMidSta[ind][5]

            data[currLineMidSta[ind][4]][currLineMidSta[ind + 1][4]] = (currHour, currMinute, abs(currCost), currDist, currLineID)
            # data[currLineMidSta[ind + 1][4]][currLineMidSta[ind][4]] = (currHour, currMinute, abs(currCost), currDist, currLineID)
        # break

for dataKey in data.keys():
    data[dataKey] = {k : v for k, v in sorted(data[dataKey].items(), key = lambda x : x[1])}

In [4]:
with open('processed_data_final.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)

In [3]:
with open('processed_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [6]:
with open('sta_cities.json', 'r', encoding='utf-8') as f:
    cities = json.load(f)

In [38]:
def fetch_wikidata(station_name: str):
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'search': station_name,
        'language': 'zh'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Ensure we catch HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [7]:
sta_city_zh = {}
with open('data/cnstation.csv', 'r', encoding='utf-8') as f:
    csv_reader = csv.reader(f)
    for ind, row in enumerate(csv_reader):
        if ind == 0:
            continue
        sta_name = row[0].split("站")[0]
        sta_city_zh[sta_name] = (row[5], row[6])

In [17]:
sta_translate = {}
sta_city = {}
sta_region = {}
zhixiashi = ["上海", "天津", "北京", "重庆"]
for sta in tqdm(data.keys()):
    sta_translate[sta] = pinyin.get(sta, format="strip")
    tempsta = sta
    while len(tempsta) >= 1 and tempsta not in sta_city_zh:
        tempsta = tempsta[:-1]
    if len(tempsta) == 0:
        print(sta)
        sta_city[sta] = pinyin.get(cities[sta], format="strip")
        sta_region[sta] = "TODO"
        continue
    if sta_city_zh[tempsta][0] in zhixiashi:
        sta_city[sta] = pinyin.get(sta_city_zh[tempsta][0], format="strip")
        sta_region[sta] = pinyin.get(sta_city_zh[tempsta][0], format="strip")
    else:
        sta_city[sta] = pinyin.get(sta_city_zh[tempsta][1], format="strip")
        sta_region[sta] = pinyin.get(sta_city_zh[tempsta][0], format="strip")
        
    # resJson = fetch_wikidata(sta)
    # if resJson is None or 'search' not in resJson:
    #     sta_translate[sta] = pinyin.get(sta)
    #     sta_city[sta] = pinyin.get(sta)
    #     print(sta)
    #     print(str(resJson))
    # else:
    #     try:
    #         currInd = 0
    #         while currInd <= len(resJson["search"]) - 1 and resJson["search"][currInd]["display"]["label"]["language"] != "en":
    #             currInd += 1
    #         if currInd >= len(resJson["search"]):
    #             sta_translate[sta] = pinyin.get(sta)
    #             sta_city[sta] = pinyin.get(sta)
    #             print(sta)
    #             print(str(resJson))
    #             continue
    #         label = resJson["search"][currInd]["label"]
    #         if "description" in resJson["search"][currInd]:
    #             description = resJson["search"][currInd]["description"]
    #         else:
    #             description = None
    #     except Exception as e:
    #         print("Error: " + str(e) + ", Json: " + str(resJson))

    #     try:
    #         if "Railway Station" in label:
    #             staEng = label.split("Railway Station")[0].strip()
    #             if description is not None:
    #                 tempDes = description.split(",")[0].strip()
    #                 print(sta, tempDes)
    #                 staCity = tempDes.split()[-1]
    #             else:
    #                 staCity = staEng
    #         elif "railway station" in label:
    #             staEng = label.split("railway station")[0].strip()
    #             if description is not None:
    #                 tempDes = description.split(",")[0].strip()
    #                 print(sta, tempDes)
    #                 tempDesSplit = tempDes.split()
    #                 if tempDesSplit[-1] == "District" or tempDesSplit[-1] == "County" or tempDesSplit[-1] == "Zoo" or tempDesSplit[-1] == "Subdistrict":
                        
    #                 staCity = tempDes.split()[-1]
    #             else:
    #                 staCity = staEng
    #         else:
    #             staEng = label
    #             staCity = label
    #         sta_translate[sta] = staEng
    #         sta_city[sta] = staCity
    #     except Exception as e:
    #         print("Station: " + sta + ", Error: " + str(e))

100%|██████████| 392/392 [00:00<00:00, 130004.52it/s]

义乌
三穗
芷江
婺源
德兴
五府山
汉口
汉川
仙游
惠安
白沟
秦皇岛
双城北
凤城东
宾阳
即墨北
牟平
荣成
介休东
高安
宜春
双峰北
怀集
恭城
广宁
云浮东
郁南
南江口
白洋淀
胜芳
福田
珲春
璧山
鲘门
弋江
繁昌西
当涂东


In [ ]:
# with open('sta_translate_v2.json', 'w', encoding='utf-8') as f:
#     json.dump(sta_translate, f, ensure_ascii=False)
# with open('sta_city_v2.json', 'w', encoding='utf-8') as f:
#     json.dump(sta_city, f, ensure_ascii=False)
# with open('sta_region_v2.json', 'w', encoding='utf-8') as f:
#     json.dump(sta_region, f, ensure_ascii=False)

In [ ]:
# with open('sta_cities.json', 'w', encoding='utf-8') as f:
#     json.dump(cities, f, ensure_ascii=False)

In [ ]:
with open('sta_cities.json', 'r', encoding='utf-8') as f:
    cities = json.load(f)

In [11]:
with open('sta_city_v2.json', 'r', encoding='utf-8') as f:
    sta_city = json.load(f)
with open('sta_region_v2.json', 'r', encoding='utf-8') as f:
    sta_region = json.load(f)
with open('sta_translate_v2.json', 'r', encoding='utf-8') as f:
    sta_translate = json.load(f)

In [12]:
cities_to_sta = {}
for sta, city in sta_city.items():
    if city not in cities_to_sta:
        cities_to_sta[city] = []
    cities_to_sta[city].append(sta)

In [13]:
regions_to_sta = {}
for sta, region in sta_region.items():
    if region not in regions_to_sta:
        regions_to_sta[region] = []
    regions_to_sta[region].append(sta)

In [14]:
cities_to_sta = {k: v for k, v in sorted(cities_to_sta.items(), key = lambda x: x[0])}

In [15]:
regions_to_sta = {k: v for k, v in sorted(regions_to_sta.items(), key = lambda x: x[0])}

In [24]:
with open('cities_sta_v2.json', 'w', encoding='utf-8') as f:
    json.dump(cities_to_sta, f, ensure_ascii=False)

In [25]:
with open('regions_sta_v2.json', 'w', encoding='utf-8') as f:
    json.dump(regions_to_sta, f, ensure_ascii=False)

In [16]:
# all_cities = list(cities_to_sta.keys())
all_regions = list(regions_to_sta.keys())
with open('processed_routes.csv', 'w', encoding='utf-8') as f:
    f.write("from_city,from_region_id,from_region_name,to_city,to_region_id,to_region_name,type,from_station,to_station,duration,distance,cost,route_id,route_name,enabled\n")
    for sta1, vtemp in data.items():
        for sta2, vals in vtemp.items():
            f.write(sta_city[sta1] + ",")
            f.write(str(all_regions.index(sta_region[sta1]) + 1000) + ",")
            f.write(sta_region[sta1] + ",")
            f.write(sta_city[sta2] + ",")
            f.write(str(all_regions.index(sta_region[sta2]) + 1000) + ",")
            f.write(sta_region[sta2] + ",")
            f.write("train,")
            f.write(sta_translate[sta1] + ",")
            f.write(sta_translate[sta2] + ",")
            f.write(str(vals[0]) + ":" + (str(vals[1]) if vals[1] >= 10 else ("0" + str(vals[1]))) + ",")
            f.write(str(vals[3]) + ",")
            f.write(str(vals[2]) + ",")
            f.write(vals[4] + "," + vals[4] + "," + "True\n")